In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils import data
from tinyimagenet import TinyImageNet
from tensorboardX import SummaryWriter
from torch.autograd import Variable


In [ ]:
from utils import show_images_horizontally
from NaiveResNet import NaiveResNet

In [ ]:
%reload_ext autoreload
%autoreload 2

## 步驟
- 讀取資料集
- 簡單 EDA
    - facets
- 定義目標 / loss function
- 定義模型
- 訓練模型
- 測試模型
- 視覺化 kernels / parameters

## 前處理資料
- 讀取資料
- 轉換（灰階處理、Augmentation、Crop）

注意在 validation 時我們不需要做 augmentation

In [ ]:
root = './dataset'

In [ ]:
# The output of torchvision datasets are PILImage images of range [0, 1]. 
# We transform them to Tensors of normalized range [-1, 1].
# normalize 在現在有 batch-normalization 的情況下其實非必要
normalize = transforms.Normalize((.5, .5, .5), (.5, .5, .5))

augmentation = transforms.RandomApply([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(64)], p=.8)

training_transform = transforms.Compose([
    transforms.Lambda(lambda x: x.convert("RGB")),
    augmentation,
    transforms.ToTensor(),
    normalize])

valid_transform = transforms.Compose([
    transforms.Lambda(lambda x: x.convert("RGB")),
    transforms.ToTensor(),
    normalize])

In [ ]:
training_set = TinyImageNet(root, 'train', transform=training_transform)
valid_set = TinyImageNet(root, 'val', transform=valid_transform)

In [ ]:
training_set

## 顯示處理後圖片
主要是顯示經過 data augmentation 的圖片。為了讓模型更 robust，我們隨機進行水平翻轉、剪裁以及旋轉的處理。在這邊顯示的圖有進行反正規化（un-normalization）。

In [ ]:
tmpiter = iter(data.DataLoader(training_set, batch_size=10, shuffle=True))
for _ in range(5):
    images, labels = tmpiter.next()
    show_images_horizontally(images, un_normalize=True)

## 定義 loss function

In [ ]:
ce_loss = nn.CrossEntropyLoss()

## 建立模型

In [ ]:
resnet = NaiveResNet(num_classes=200)

In [ ]:
dummy_input = Variable(torch.rand(16, 3, 64, 64))
out = resnet.forward(dummy_input)
out.size()

將模型圖寫到 Tensorboard 以供確認

In [ ]:
with SummaryWriter(comment='NaiveResNet') as w:
    w.add_graph(resnet, (dummy_input, ))

## 定義 Optimizer

In [ ]:
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)

## 訓練模型

In [ ]:
params = {'batch_size': 64, 'shuffle': True, 'num_workers': 6}
# params = {'batch_size': 2, 'shuffle': True, 'num_workers': 1}
max_epochs = 1

In [ ]:
trainloader = data.DataLoader(training_set, **params)
# validloader = data.DataLoader(valid_set, )

In [ ]:
%%time
for epoch in range(max_epochs):
    epoch_loss = 0.0
    for idx, (data, target) in enumerate(trainloader):
        optimizer.zero_grad()
        output = resnet(data)
        batch_loss = ce_loss(output, target)
        batch_loss.backward()
        optimizer.step()
        epoch_loss += batch_loss.item()
        print(idx, batch_loss)
    
    epoch_loss /= len(trainloader)
    print(epoch_loss)
        